In [2]:
# %load Main.py
import numpy as np
import tensorflow as tf
import PreProcess as PrePro
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

from keras.optimizers import Adam, SGD,RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from keras.models import load_model

import sklearn
import math
import unet_model
import Data_Load
import unet_info
import common
import pandas as pd
import logging
import argparse
import time
import segnet
import maskrcnn
import os
import openpyxl

Using TensorFlow backend.


In [3]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
unet_info.begin()

GPU setting completed.
Tensorflow version: 1.10.0
Keras version 2.2.0
Dim ordering: tf


In [4]:
subseq =4096
dbName  = "SepsisDataset"
print("Load data")
X, Y, X_train, X_test, Y_train, Y_test, N_FEATURES, classes = Data_Load.load_data(dbName,subseq)
#get dense prediction results with overlap(transformed win data label's ground truth)
# y_test_resh = y_test.reshape(y_test.shape[0], y_test.shape[2], -1)
# y_test_resh_argmax = np.argmax(y_test_resh, axis=2)
# labels_test_unary = y_test_resh_argmax.reshape(y_test_resh_argmax.size)
# file_labels_test_unary = 'labels_gd_'+args.dataset+'_'+str(subseq)+'0317.npy'
# np.save(file_labels_test_unary,labels_test_unary)
print("Finish Load data")

Load data
(200000, 1, 4096, 1)
Finish Load data


In [5]:
## 10-flod cross validation###
del X,Y
epochs = 45 #50
batch_size = 64 #64 #128
optim_type = 'adam'
learning_rate = 0.02 # 0.05 # 0.01 #0.02
sum_time = 0
net = 'unet'
block = '5'

In [12]:
rf_path = 'Result/4096/0321_0205_4096'
nowtime = '0321_0205_4096'
cvscores = []
cvsloss = []

In [ ]:
ksize = 10
pice = int(X_train.shape[0]/ ksize)
print('k-fold Floder created') if os.path.isdir(rf_path +'/k_flod') else os.mkdir(rf_path+'/k_flod')
for k in range(6):
    fp = open(rf_path+'/Result_'+nowtime+'.txt','a')
    trainX = X_train[np.r_[0:k*pice,(k+1)*pice:X_train.shape[0]]]
    trainY = Y_train[np.r_[0:k*pice,(k+1)*pice:X_train.shape[0]]]
    valX = X_train[np.r_[k*pice:(k+1)*pice]]
    valY = Y_train[np.r_[k*pice:(k+1)*pice]]
    print("\n")
    print(str(k)+" Running...")
    print(str(k)+" Running...",file=fp)
    # load model
    Kmodel = load_model(rf_path+ '/k_flod/'+str(k)+'_Model_0321_0205_4096_64_adam.h5py')
    
    Kscores = Kmodel.evaluate(valX, valY, verbose=0)
    print("%s: %.2f%%" % (Kmodel.metrics_names[1], Kscores[1]*100), file=fp)
    print("%s: %.2f%%" % (Kmodel.metrics_names[0], Kscores[0]), file=fp)
    cvscores.append(Kscores[1])
    cvsloss.append(Kscores[0])
    print(cvscores)
    print(cvsloss)
    fp.close()
    
    del trainX,trainY, valX, valY, Kmodel

k-fold Floder created


0 Running...
[0.9941199951171875]
[0.016468735366262435]


1 Running...
[0.9941199951171875, 0.9937726915147569]
[0.016468735366262435, 0.01558910991894825]


2 Running...
[0.9941199951171875, 0.9937726915147569, 0.993624267578125]
[0.016468735366262435, 0.01558910991894825, 0.017197023864660068]


3 Running...
[0.9941199951171875, 0.9937726915147569, 0.993624267578125, 0.9909441731770834]
[0.016468735366262435, 0.01558910991894825, 0.017197023864660068, 0.03487152868556344]


4 Running...
[0.9941199951171875, 0.9937726915147569, 0.993624267578125, 0.9909441731770834, 0.9949945339626736]
[0.016468735366262435, 0.01558910991894825, 0.017197023864660068, 0.03487152868556344, 0.015518172406586624]


5 Running...
